In [1]:
# 네이버 맞춤법 교정 라이브러리 사용 위한 패키지 설치
!pip install git+https://github.com/ssut/py-hanspell.git # --target=$my_path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-43boex4y
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-43boex4y


In [2]:
# 띄어쓰기 교정 위한 라이브러리 사용 위한 패키지 설치
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-jlh597nh
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-jlh597nh
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [3]:
# 형태소 분석기 사용 위한 패키지 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
# 데이터를 다루는 가장 기본이 되는 라이브러리
import pandas as pd
# 데이터를 엑셀의 형태로 보여주는 라이브러리
from nltk.corpus import stopwords
# 불용어 삭제 라이브러리
from pykospacing import Spacing
# 띄어쓰기 교정 라이브러리
import nltk
# 자연어 처리를 위한 라이브러리
from hanspell import spell_checker
# 네이버 한글 맞춤법 검사기 기반 패키지
from tqdm.auto import tqdm
# 
import copy

In [6]:
# 데이터 로드
train_data = pd.read_csv('/content/drive/MyDrive/산업인공지능/박시우/raw_train.CSV', encoding = 'cp949',index_col = 0)
test_x = pd.read_csv('/content/drive/MyDrive/산업인공지능/박시우/raw_test.csv', encoding = 'cp949')

In [8]:
# 데이터 할당
train_x = train_data['문장'][:12158]# okt로 인한 메모리 초과 막기 위해 train_data 일부 자르고 가져오기
test_x = test_x['문장']

In [9]:
# 데이터 량 
row_train, = np.shape(train_x)
row_test, = np.shape(test_x)

In [10]:
# 문장 속 특수기호 삭제
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~[]" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, "")
    
    for p in punct:
        text = text.replace(p, '')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, '')
    
    return text.strip()
#train data 문장
for i in range(row_train):
  train_x[i] = clean_punc(str(train_x[i]), punct, punct_mapping)
#test data 문장
for i in range(row_test):
  test_x[i] = clean_punc(str(test_x[i]), punct, punct_mapping)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
train_x

0
0                                   일안하는 시간은 쉬고싶어서 그런게 아닐까
1        아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다 무슬림 50퍼 근친이다 10...
2        루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵곡...
3        홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...
4        아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다 여자들은 더 심하게 그런다...
                               ...                        
12153    똥꼬충들은 에이즈 전파 주범이므로 질병관리 예방 차원에서라도연쇄살인마에 준하는 형벌...
12154                              문재인찍고 토착왜구 물러나라 잘하게보이던데
12155                            굳세워라 갓건배모든 남혐유튜버 아묻따 응원한다
12156                         국제결혼도 한몫한다봄 똥남아여자랑 결혼을 왜하는거냐
12157                                                와꾸한남콘
Name: 문장, Length: 12158, dtype: object

In [12]:
train_x[0]

'일안하는 시간은 쉬고싶어서 그런게 아닐까'

In [13]:
test_x

0       ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...
1                                여자들은 취미가 애낳는건가 취미를 좀 가져라
2                                개슬람녀 다 필요없고 니 엄마만 있으면 된다
3       조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...
4                                   바지 내리다 한남들 와꾸 보고 올려뿟노
                              ...                        
3732                          저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ
3733                               다른것보다 눈이 어떻게 저렇게 생기노탄식
3734                 막노동을 해도 한국에살고말지 미쳤다고 남미를가냐차라리 자살을하겠다
3735    사형을 구형하였으나 여성인 점을 감안해 25년 선고 내가 뭐 잘못본건가개시발 보지민...
3736                              참군남이노 남자의 용도는 고기방패다 ㅋㅋㅋ
Name: 문장, Length: 3737, dtype: object

In [14]:
# 띄어쓰기 교정
spacing = Spacing()
for i in range(row_train):
  train_x[i] = spacing(train_x[i])
for i in range(row_test):
  test_x[i] = spacing(test_x[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
# 맞춤법 교정
for i in tqdm(range(row_train)):
  try:
    result_train = spell_checker.check(train_x[i])
    train_x[i] = result_train.as_dict()['checked']
  except:
    pass
for i in tqdm(range(row_test)):
  try:
    result_test = spell_checker.check(test_x[i])
    test_x[i] = result_test.as_dict()['checked']
  except:
    pass

  0%|          | 0/12158 [00:00<?, ?it/s]

  0%|          | 0/3737 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [16]:
# 문장 속 특수기호 삭제
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~[]" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, "")
    
    for p in punct:
        text = text.replace(p, '')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, '')
    
    return text.strip()


In [17]:
#train data 문장 - 특수기호 삭제
for i in range(row_train):
  train_x[i] = clean_punc(str(train_x[i]), punct, punct_mapping)

# test data 문장 - 특수기호 삭제
for i in range(row_test):
  test_x[i] = clean_punc(str(test_x[i]), punct, punct_mapping)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [18]:
# StopWord Removal(불용어 제거) - 숫자
import re
p = re.compile("[0-9]+")

for i in range(row_train):
  try:
    train_x[i] = p.sub("", train_x[i])
  except:
    train_x[i] = '름'

for i in range(row_test):
  try:
    test_x[i] = p.sub("", test_x[i])
  except:
    test_x[i] = '름'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [19]:
from konlpy.tag import Okt
Okt = Okt()

In [20]:
Okt.pos(train_x[0])

[('일', 'Noun'),
 ('안', 'Noun'),
 ('하는', 'Verb'),
 ('시간', 'Noun'),
 ('은', 'Josa'),
 ('쉬', 'Noun'),
 ('고', 'Josa'),
 ('싶어서', 'Verb'),
 ('그런', 'Adjective'),
 ('게', 'Noun'),
 ('아닐까', 'Adjective')]

In [21]:
okt_pos_train_x = copy.deepcopy(train_x)
okt_pos_test_x = copy.deepcopy(test_x)
# 독립된 객체를 만들기 위한 깊은 복사

In [22]:
for i in range(row_train):
  okt_pos_train_x[i] = Okt.pos(okt_pos_train_x[i])
for i in range(row_test):
  okt_pos_test_x[i] = Okt.pos(okt_pos_test_x[i])

In [23]:
okt_nojosa_train_x = copy.deepcopy(okt_pos_train_x)
okt_nojosa_test_x = copy.deepcopy(okt_pos_test_x)

In [24]:
for i in range(row_train):
  j=0
  num = len(okt_nojosa_train_x[i])
  while j < num:
    x, y = okt_nojosa_train_x[i][j]
    if y not in ['Noun', 'Adjective', 'Verb']:
      del okt_nojosa_train_x[i][j]
      num -= 1
    else:
      j += 1


In [25]:
for i in range(row_test):
  j=0
  num = len(okt_nojosa_test_x[i])
  while j < num:
    x, y = okt_nojosa_test_x[i][j]
    if y not in ['Noun', 'Adjective', 'Verb']:
      del okt_nojosa_test_x[i][j]
      num -= 1
    else:
      j += 1

In [26]:
nojosa_test_x = okt_nojosa_test_x.astype('string')
nojosa_train_x = okt_nojosa_train_x.astype('string')

In [27]:
# 문장 속 특수기호 삭제
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~[]" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, "")
    
    for p in punct:
        text = text.replace(p, '')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, '')
    
    return text.strip()

# train data 문장 
for i in range(row_train):
  nojosa_train_x[i] = clean_punc(nojosa_train_x[i], punct, punct_mapping)

#test data 문장
for i in range(row_test):
  nojosa_test_x[i] = clean_punc(str(nojosa_test_x[i]), punct, punct_mapping)

In [28]:
for i in range(row_train):
  nojosa_train_x[i] = re.sub('[^가-힣]', ' ', nojosa_train_x[i])

In [29]:
for i in range(row_test):
  nojosa_test_x[i] = re.sub('[^가-힣]', ' ', nojosa_test_x[i])

In [30]:
nojosa_train_x

0
0        일      안      하는      시간      쉬      싶어서      ...
1        아동      성범죄      폐      버      기록      바      ...
2        루나      솔로      앨범      나왔을      때      메모    ...
3        홍      박      어버이연합      보내요      뭐      이런   ...
4        놔      왜      여기      댓글      여자      김치      ...
                               ...                        
12153    똥      고충      에이즈      전파      범      므      ...
12154    문재인      찍고      토착      왜구      물러나라      하게 ...
12155    굳           세워라      갓      건배      모든      남 ...
12156    국제      결혼      한몫      봄      똥      남아      ...
12157                                           와꾸한남콘     
Name: 문장, Length: 12158, dtype: string

In [31]:
nojosa_test_x

0       진짜      죽어도      상관없다는           마인드      싸웠더니...
1       여자      취미      애      낳는      건      가      취...
2       개슬람녀      필요      없고           네      엄마      ...
3       조팔      남한      길거리      돌아다니면      한국      남자...
4       바지      리다      한남      와      꾸      보고      ...
                              ...                        
3732    저      시대      언어      면      한      남충      시...
3733    다른      것      눈      어떻게           생기      노 ...
3734    막노동      해도      한국      살      말      미쳤다고   ...
3735    사형      구형      하였으나      여성      점      감안   ...
3736    참      군      남이노      남자      용도      고기     ...
Name: 문장, Length: 3737, dtype: string